In [74]:
import os
import re
import json
import glob

target_folder = "./split_pdfs"

pattern = re.compile(r"조사료-단면_(\d+)_(\d+)\.json")

file_infos = []

# 대상 폴더에서 파일 추출
for file_path in glob.glob(os.path.join(target_folder, "*.json")):
    file_name = os.path.basename(file_path)
    match = pattern.match(file_name)
    if match:
        start_page = int(match.group(1))
        file_infos.append((start_page, file_path))

# 페이지 번호 기준 정렬
file_infos.sort(key=lambda x: x[0])

# 병합된 HTML 문자열 준비
merged_html = ""

# 병합 수행
for _, file_path in file_infos:
    with open(file_path, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            html = data["content"]["html"]
            merged_html += html + "\n"
        except Exception as e:
            print(f"❌ 오류: {file_path} → {e}")

print("✅ 병합 완료. 총 길이:", len(merged_html))


✅ 병합 완료. 총 길이: 426828


In [2]:
merged_html

'<header id=\'0\' style=\'font-size:16px\'>발간등록번호 11-1390000-004529-14<br>농업기술길잡이 126</header>\n<br><table id=\'1\' style=\'font-size:14px\'><thead><tr><td>OPEN</td><td>출차표시 변경금지</td></tr></thead><tbody><tr><td>공공누리</td><td>공공저작물 자유이몸허락</td></tr></tbody></table>\n<h1 id=\'2\' style=\'font-size:22px\'>두</h1>\n<br><figure id=\'3\'><img alt="" data-coord="top-left:(487,648); bottom-right:(773,980)" /><figcaption id=\'4\' style=\'font-size:18px\'>무는 우리나라 주요 부식인 김치의 주재료로서,<br>재배 면적당 생산액이 높은 소득 작물 중<br>하나이다. 김치 외에도 조림 및 국거리,<br>무말랭이, 시래기 등 부위에 따라 여러 용도로<br>사용되고 있다.</figcaption></figure>\n\n<p id=\'5\' data-category=\'paragraph\' style=\'font-size:20px\'>농촌진흥청</p>\n<h1 id=\'6\' style=\'font-size:18px\'>농업기술길잡이 목록</h1>\n<table id=\'7\' style=\'font-size:14px\'><thead><tr><td>가지</td><td>무</td><td>올리브</td></tr></thead><tbody><tr><td>가축분뇨처리와 자원화</td><td>무화과</td><td>올바른 비료사용법</td></tr><tr><td>가축위생과 질병(기타 가축질병편)</td><td>밀</td><td>우리 꽃 기르기</td></tr><tr><td>가축위생과 질병(닭)</td><td>밭농사기계</td><td>우리가 알아야 할

In [17]:
data["content"]["html"]

'<h1 id=\'0\' style=\'font-size:20px\'>전라북도</h1>\n<h1 id=\'1\' style=\'font-size:16px\'>가. 김병귀 씨 농가</h1>\n<br><p id=\'2\' data-category=\'paragraph\' style=\'font-size:18px\'>천지원 농장(대표 김병귀)은 26ha(시설 13ha, 노지 13ha)의 면적에서 쌈 채소<br>류를 생산해 호남 지역의 롯데마트, 농협 하나로마트 등 대형 유통업체에 출하하<br>고, 철저한 위생 관리를 통한 채소류로 학교급식 사업을 실시하고 있다. 유기농업<br>을 통한 농업인이 되고자 했던 것은 1990년, 4년여 동안의 원양어선 생활을 마치<br>고 고향으로 돌아왔을 때였다. 고혈압과 각종 질병으로 고통받았을 때 좋다는 약<br>은 다 먹어 보았으나 별 효과를 보지 못했는데, 우연히 알게 된 자연식을 통한 자<br>연건강법으로 기적과 같이 효과를 보게 되었고, 그때의 깨달음으로 모든 사람에<br>게 유익을 줄 수 있는 유기농산물을 생산해보아야겠다고 결심하고 귀농을 하게<br>되었다. 처음 5년여 동안은 농사에 대한 지식과 경험 부족으로 많은 실패와 재<br>정난에 처하기도 했다. 그러나 당시의 힘들었던 상황은 농사에 대한 막대한 학<br>습비를 지불한 것이라고 생각하고 실패를 통한 기술 습득을 발판으로 다시 한번<br>제2의 시작을 해 보아야겠다는 결심을 하게 되었다.<br>김병기 씨는 상추를 재배하고자 한다면 많은 생각을 해야하는데 그 중 품종 선택<br>에서 몇 가지 고려해야 한다고 한다.<br>① 먼저 용도를 생각해 포기상추로 할 것인지, 치마상추로 할 것인지를 판단한다.<br>② 계절에 따라 겨울 재배용인지, 혹서기를 고려해 추대가 안정된 여름 재배용인<br>지를 판단한다.<br>③ 쌈용으로 할 것인지, 샐러드용으로 할 것인지를 판단한다.<br>④ 당도와 아삭한 정도, 기호도 등에 따라 어떤 품종을 선택할지를 판단한다.<br>그 후 비닐하우

In [75]:
from bs4 import BeautifulSoup
import re


# 이제 HTML 전체를 파싱
soup = BeautifulSoup(merged_html, 'html.parser')

chunks = []
current_title = None
current_content = ""

for tag in soup.find_all(['h1', 'p', 'table']):
    text = tag.get_text(separator="\n").strip()

    if tag.name == 'h1':
        style = tag.attrs.get('style', '')
        font_size_match = re.search(r'font-size\s*:\s*(\d+)px', style)
        
        if font_size_match:
            font_size = int(font_size_match.group(1))
            if font_size >= 20:
                if current_title and current_content:
                    chunks.append({"title": current_title, "content": current_content.strip()})
                    current_content = ""
                current_title = text
            else:
                current_content += f"\n{text}"
    elif tag.name in ['p', 'table']:
        current_content += f"\n{text}"

# 마지막 chunk 추가
if current_title and current_content:
    chunks.append({"title": current_title, "content": current_content.strip()})


In [76]:
chunks

[{'title': '조사료',
  'content': '발간등록번호\n11-1390000-004953-14\nOPEN\n채표시 변경금지\n공공누리\n공공저작물 자유이용허락\n농업기술길잡이 91'},
 {'title': '대표집필부서', 'content': '국립축산과학원 초지사료과 041-580-6741'},
 {'title': '집필진',
  'content': '제1장 조사료 일반현황\n이상훈 국립축산과학원 초지사료과\n박형수 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n제2장 사료작물의 품종 개발\n이상훈 국립축산과학원 초지사료과\n이기원 국립축산과학원 초지사료과\n우제훈 국립축산과학원 초지사료과\n최보람 국립축산과학원 초지사료과\n임은아 국립축산과학원 초지사료과\n제3장 사료작물 재배기술\n박형수 국립축산과학원 초지사료과\n최기춘 국립축산과학원 초지사료과\n양승학 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n최보람 국립축산과학원 초지사료과\n제4장 사료작물 연중 심기차례\n박형수 국립축산과학원 초지사료과\n최기춘 국립축산과학원 초지사료과\n양승학 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n제5장 조사료 제조 및 이용\n박형수 국립축산과학원 초지사료과\n최기춘 국립축산과학원 초지사료과\n양승학 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n우제훈 국립축산과학원 초지사료과\n제6장 곤포 사일리지 제조 장비\n박형수 국립축산과학원 초지사료과\n최기춘 국립축산과학원 초지사료과\n양승학 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n우제훈 국립축산과학원 초지사료과\n제7장 초지 조성과 관리\n박형수 국립축산과학원 초지사료과\n최기춘 국립축산과학원 초지사료과\n양승학 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n제8장 친환경 조사료 생산과 이용\n박형수 국립축산과학원 초지사료과\n최기춘 국립축산과학원 초지사료과\n양승학 국립축산과학원 초지사료과\n정종성 국립축산과학원 

In [77]:
import time
import uuid

today = time.strftime('%Y-%m-%d')
doc_id = time.strftime('%Y-%m-%d_')+str(uuid.uuid4())[:10]

for i, chunk in enumerate(chunks):
    chunk["chunk_id"] = f"{doc_id}_{i:03}"
    chunk["date"] = today
    chunk["document"] = "조사료-단면.pdf"
    
chunks

[{'title': '조사료',
  'content': '발간등록번호\n11-1390000-004953-14\nOPEN\n채표시 변경금지\n공공누리\n공공저작물 자유이용허락\n농업기술길잡이 91',
  'chunk_id': '2025-07-18_dd0e4ffd-2_000',
  'date': '2025-07-18',
  'document': '조사료-단면.pdf'},
 {'title': '대표집필부서',
  'content': '국립축산과학원 초지사료과 041-580-6741',
  'chunk_id': '2025-07-18_dd0e4ffd-2_001',
  'date': '2025-07-18',
  'document': '조사료-단면.pdf'},
 {'title': '집필진',
  'content': '제1장 조사료 일반현황\n이상훈 국립축산과학원 초지사료과\n박형수 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n제2장 사료작물의 품종 개발\n이상훈 국립축산과학원 초지사료과\n이기원 국립축산과학원 초지사료과\n우제훈 국립축산과학원 초지사료과\n최보람 국립축산과학원 초지사료과\n임은아 국립축산과학원 초지사료과\n제3장 사료작물 재배기술\n박형수 국립축산과학원 초지사료과\n최기춘 국립축산과학원 초지사료과\n양승학 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n최보람 국립축산과학원 초지사료과\n제4장 사료작물 연중 심기차례\n박형수 국립축산과학원 초지사료과\n최기춘 국립축산과학원 초지사료과\n양승학 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n제5장 조사료 제조 및 이용\n박형수 국립축산과학원 초지사료과\n최기춘 국립축산과학원 초지사료과\n양승학 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n우제훈 국립축산과학원 초지사료과\n제6장 곤포 사일리지 제조 장비\n박형수 국립축산과학원 초지사료과\n최기춘 국립축산과학원 초지사료과\n양승학 국립축산과학원 초지사료과\n정종성 국립축산과학원 초지사료과\n우제훈 국립축

In [78]:
import boto3

# MinIO 설정
minio_endpoint = "http://artifact-store:9000"  # 도커 내부라면 'host.docker.internal:9000'
access_key = "minio"
secret_key = "miniostorage"

# S3 클라이언트 생성 (endpoint_url 반드시 지정)
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
)


In [79]:
import io 
import json 

buffer = io.BytesIO()
buffer.write(json.dumps(chunks, ensure_ascii=False).encode("utf-8"))
buffer.seek(0)

# 저장 파일명
filename = f"data/조사료-단면_{doc_id}.json"

s3_client.upload_fileobj(buffer, "chunk", filename)
print("업로드 완료")

업로드 완료


In [80]:
import os
import glob
import json
import io
import uuid
from datetime import datetime

target_folder = "./split_pdfs"
upload_bucket = "ocr-raw"

# 파일 리스트 가져오기
file_paths = sorted(glob.glob(os.path.join(target_folder, "*.json")))

for file_path in file_paths:
    file_name = os.path.basename(file_path)

    # 특정 이름이 들어간 파일만 처리
    if "조사료-단면" not in file_name:
        continue

    print(f"업로드 중: {file_name}")

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        buffer = io.BytesIO()
        buffer.write(json.dumps(data, ensure_ascii=False).encode("utf-8"))
        buffer.seek(0)

        s3_key = f"raw_data/{str(uuid.uuid4())}_{datetime.now().strftime('%Y-%m-%d')}_{file_name}"
        s3_client.upload_fileobj(buffer, upload_bucket, s3_key)

        print(f"업로드 완료: {file_name}")

    except Exception as e:
        print(f"업로드 실패: {file_name} → {e}")


업로드 중: 조사료-단면_121_150.json
✅ 업로드 완료: 조사료-단면_121_150.json
업로드 중: 조사료-단면_151_180.json
✅ 업로드 완료: 조사료-단면_151_180.json
업로드 중: 조사료-단면_181_210.json
✅ 업로드 완료: 조사료-단면_181_210.json
업로드 중: 조사료-단면_1_30.json
✅ 업로드 완료: 조사료-단면_1_30.json
업로드 중: 조사료-단면_211_240.json
✅ 업로드 완료: 조사료-단면_211_240.json
업로드 중: 조사료-단면_241_270.json
✅ 업로드 완료: 조사료-단면_241_270.json
업로드 중: 조사료-단면_271_300.json
✅ 업로드 완료: 조사료-단면_271_300.json
업로드 중: 조사료-단면_301_330.json
✅ 업로드 완료: 조사료-단면_301_330.json
업로드 중: 조사료-단면_31_60.json
✅ 업로드 완료: 조사료-단면_31_60.json
업로드 중: 조사료-단면_331_342.json
✅ 업로드 완료: 조사료-단면_331_342.json
업로드 중: 조사료-단면_61_90.json
✅ 업로드 완료: 조사료-단면_61_90.json
업로드 중: 조사료-단면_91_120.json
✅ 업로드 완료: 조사료-단면_91_120.json
